## 迁移学习

在深度学习实践中，我们很少有人会从头开始训练整个卷积网络（随机初始化权重），因为要从头训练一个卷积网络是需要足够多的数据的，而通常数据非常难获取。**相反，我们直接下载别人已经训练好的某个网络结构的权重，用这个作为预训练，然后通过简单的微调模型切换到你感兴趣的任务上，将会进展的非常快。**大多数的Kaggle竞赛的解决方案都是如此。

比如ImageNet，或者MS COCO，或者Pascal类型的数据集，这些都是不同数据集的名字，它们都是由大家上传到网络的，并且有大量的计算机视觉研究者已经用这些数据集训练过他们的算法了。有时候这些训练过程需要花费好几周，并且需要很多的GPU，其它人已经做过了，并且经历了非常痛苦的寻最优过程，**这就意味着你可以下载花费了别人好几周甚至几个月而做出来的开源的权重参数，把它当作一个很好的初始化用在你自己的神经网络上。**

假设有一个在ImageNet数据集（140万张标记图像，1000个不同的类别）上训练好的大型卷积神经网络 ImageNet中包含许多动物类别，其中包括不同种类的蚂蚁和蜜蜂，因此可以认为它在蚂蚁和蜜蜂分类问题上也能有良好的表现。

#### 目前，主要两种迁移学习方法如下：

- **特征提取（feature extraction）**：特征提取是使用之前网络学到的表示来从新样本中提取出有趣的特征。然后将这些特征输入一个新的分类器，从头开始训练。
用于图像分类的卷积神经网络包含两部分：首先是一系列池化层和卷积层，最后是一个密集连接分类器。第一部分叫作模型的卷积基（convolutional base）。对于卷积神经网络而言，特征提取就是取出之前训练好的网络的卷积基，在上面运行新数据，然后在输出上面练一个新的分类器。

![](feature_extraction.png)

**我们将冻结除最后一个全连接层之外的所有网络的权重。最后一个全连接层被替换为具有随机权重的新层，并且仅训练该层。**举个列子，用一个在ImageNet上预先训练的卷积层，删除最后一个完全连接的层(该层的输出是ImageNet等不同任务的1000个类别的得分)，然后将ConvNet的其余部分作为新数据集的固定特征提取器。

- **微调模型（fine-tuning）**：与特征提取互为补充。**对于用于特征提取的冻结的模型基，微调是指将其顶部的几层“解冻”，并将这解冻的几层和新增加的部分（本例中是全连接分类器）联合训练**,之所以叫作微调，是因为它只是略微调整了所复用模型中更加抽象的表示，以便让这些表示与手头的问题更加相关。

使用预训练网络权重初始化网络，而不是随机初始化，就像在ImageNet 1000数据集上训练的网络一样。举个例子，ImageNet数据集，它有1000个不同的类别，因此这个网络会有一个Softmax单元，它可以输出1000个可能类别之一。你可以去掉这个 Softmax层，创建你自己的Softmax单元，比如你想识别“猫还是狗”，那只有2个类别，可以把最后一层softmax输出从1000替换成2。通常这样你的网络只训练最后一层softmax层，而其他层都会被冻结(freeze)，也就是不参与训练，这样你的网络会获得非常好的性能，即使在一个非常小的数据集下。

In [11]:
import torch
import torch.nn as nn
import torchvision

### 微调模型（fine-tuning）
使用预训练模型，并随机初始化最后一个全连接层。

核心代码如下：

In [2]:
# torchvision工具中包含了很多预训练模型，这里直接用resnet18模型，注意pretrained=True
model_ft = torchvision.models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to C:\Users\Administrator/.torch\models\resnet18-5c106cde.pth
46827520it [00:43, 1080533.68it/s]


In [9]:
for name,param in model_ft.named_parameters():
    print("name:{}\tsize:{}".format(name,param.size()))

name:conv1.weight	size:torch.Size([64, 3, 7, 7])
name:bn1.weight	size:torch.Size([64])
name:bn1.bias	size:torch.Size([64])
name:layer1.0.conv1.weight	size:torch.Size([64, 64, 3, 3])
name:layer1.0.bn1.weight	size:torch.Size([64])
name:layer1.0.bn1.bias	size:torch.Size([64])
name:layer1.0.conv2.weight	size:torch.Size([64, 64, 3, 3])
name:layer1.0.bn2.weight	size:torch.Size([64])
name:layer1.0.bn2.bias	size:torch.Size([64])
name:layer1.1.conv1.weight	size:torch.Size([64, 64, 3, 3])
name:layer1.1.bn1.weight	size:torch.Size([64])
name:layer1.1.bn1.bias	size:torch.Size([64])
name:layer1.1.conv2.weight	size:torch.Size([64, 64, 3, 3])
name:layer1.1.bn2.weight	size:torch.Size([64])
name:layer1.1.bn2.bias	size:torch.Size([64])
name:layer2.0.conv1.weight	size:torch.Size([128, 64, 3, 3])
name:layer2.0.bn1.weight	size:torch.Size([128])
name:layer2.0.bn1.bias	size:torch.Size([128])
name:layer2.0.conv2.weight	size:torch.Size([128, 128, 3, 3])
name:layer2.0.bn2.weight	size:torch.Size([128])
name:layer

In [12]:
# 取得全连接层的输入单元个数
num_ftrs = model_ft.fc.in_features

# 微调模型，替换全连接层:把全连接层的输出神经元个数改为2
model_ft.fc = nn.Linear(num_ftrs, 2)

### 特征提取（feature extraction）
在这里，我们需要冻结除最后一层之外的所有网络。我们需要设置冻结参数requires_grad == False,以便不计算反向传播backward()的时候不计算梯度。

核心代码如下：

In [13]:
# 直接使用torchvison的预训练模型，这里可换成resnet34,resnet50,vgg16等ImageNet预训练模型
model_conv = torchvision.models.resnet18(pretrained=True)

# 冻结除最后一层之外的所有网络
for param in model_conv.parameters():
    param.requires_grad = False

# 取得全连接层的输入单元个数
# 把全连接层的输出神经元个数改为2
# 默认情况下新层的requires_grad=True
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

如果你能熟练的掌握迁移学习，那么深度学习领域中计算机识别的问题都会变得简单，可以尝试做一些简单的入门级别的kaggle竞赛了。

## 完整代码